<a href="https://colab.research.google.com/github/MichaelHuang05/test-repo/blob/master/FintechDR_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import cv2
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json

from google.colab import drive
drive.mount('/content/Drive')


path = "/content/Drive/My Drive/Colab Notebooks/images"
os.chdir(path) 
with open('/content/Drive/My Drive/Colab Notebooks/labels.json') as label:
    labels = json.load(label)

label_df = pd.DataFrame(labels, columns = ["file_name", "DR_stage"])


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/Drive


In [2]:
#### training and testing set split
from sklearn.model_selection import train_test_split 
from keras.utils import to_categorical


y_train, y_test = train_test_split(label_df, test_size=0.2)

# Onehot encoding for catogory
y_trainOneHot = to_categorical(y_train.iloc[:,1])
y_testOneHot = to_categorical(y_test.iloc[:,1])

# training set summary statistics
q = pd.DataFrame(y_train)
q.groupby(['DR_stage']).agg({'DR_stage':"count"})
# d



Using TensorFlow backend.


,DR_stage
DR_stage,
0,871
1,93
2,191
3,28
4,17


In [0]:
# Train set
################################################

train_filenames_df = pd.DataFrame(y_train.iloc[:,0])
train_images = []
train_filenames_ls = list(train_filenames_df["file_name"])

for i in range(len(train_filenames_ls)):              
    temp = cv2.imread(path + "/" + train_filenames_ls[i])
    temp = cv2.resize(temp, (512,512))
    train_images.append(temp)

train_images_arr = np.array(train_images)

# Test set
#############################################

test_images = []
test_filenames_ls = list(y_test["file_name"])

for i in range(len(test_filenames_ls)):              
    temp = cv2.imread(path + "/" + test_filenames_ls[i])
    temp = cv2.resize(temp, (512,512))
    test_images.append(temp)

test_images_arr = np.array(test_images)


In [37]:
### CNN 
import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D




model = Sequential()

model.add(Conv2D(filters = 10, kernel_size = (3,3), input_shape = (512,512,3), activation = "relu"))
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(Conv2D(filters = 10, kernel_size = (3,3), activation = "relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters = 10, kernel_size = (3,3), activation = "relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters = 10, kernel_size = (3,3), activation = "relu"))
model.add(MaxPooling2D(pool_size=(2,2)))



model.add(Flatten())
model.add(Dense(32, activation = "relu"))

model.add(Dropout(0.4))

model.add(Dense(32, activation = "relu"))

model.add(Dropout(0.4))





model.add(Dense(5, activation = 'softmax'))

#################
#model = tf.contrib.tpu.keras_to_tpu_model(
#    model,
#    strategy=tf.contrib.tpu.TPUDistributionStrategy(
#        tf.contrib.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])))
#################

model.compile(loss = "categorical_crossentropy",
             optimizer = "adam",
             metrics = ['accuracy'])

# maybe should change to focal loss?





model.summary()


Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_36 (Conv2D)           (None, 510, 510, 10)      280       
_________________________________________________________________
max_pooling2d_36 (MaxPooling (None, 255, 255, 10)      0         
_________________________________________________________________
conv2d_37 (Conv2D)           (None, 253, 253, 10)      910       
_________________________________________________________________
max_pooling2d_37 (MaxPooling (None, 126, 126, 10)      0         
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 124, 124, 10)      910       
_________________________________________________________________
max_pooling2d_38 (MaxPooling (None, 62, 62, 10)        0         
_________________________________________________________________
conv2d_39 (Conv2D)           (None, 60, 60, 10)      

In [38]:
model.fit(x = train_images_arr, y = y_trainOneHot, batch_size = 100, epochs = 1000, validation_split = 0.1, verbose = 2)

Train on 1080 samples, validate on 120 samples
Epoch 1/1000
 - 5s - loss: 4.6505 - acc: 0.6343 - val_loss: 4.3379 - val_acc: 0.6333
Epoch 2/1000
 - 3s - loss: 2.1828 - acc: 0.5630 - val_loss: 1.2139 - val_acc: 0.6833
Epoch 3/1000
 - 3s - loss: 1.3188 - acc: 0.6926 - val_loss: 1.1900 - val_acc: 0.6833
Epoch 4/1000
 - 3s - loss: 1.1652 - acc: 0.7037 - val_loss: 1.0512 - val_acc: 0.6833
Epoch 5/1000
 - 3s - loss: 1.0929 - acc: 0.6963 - val_loss: 1.0606 - val_acc: 0.6833
Epoch 6/1000
 - 3s - loss: 1.0853 - acc: 0.6907 - val_loss: 1.0283 - val_acc: 0.6833
Epoch 7/1000
 - 3s - loss: 1.0168 - acc: 0.7130 - val_loss: 0.9978 - val_acc: 0.6833
Epoch 8/1000
 - 3s - loss: 1.0046 - acc: 0.7093 - val_loss: 1.0297 - val_acc: 0.6833
Epoch 9/1000
 - 3s - loss: 1.0063 - acc: 0.7204 - val_loss: 0.9950 - val_acc: 0.6833
Epoch 10/1000
 - 3s - loss: 0.9688 - acc: 0.7120 - val_loss: 0.9775 - val_acc: 0.6833
Epoch 11/1000
 - 3s - loss: 0.9648 - acc: 0.7009 - val_loss: 1.0051 - val_acc: 0.6833
Epoch 12/1000
 -

In [39]:
scores = model.evaluate(test_images_arr, y_testOneHot)
scores[1]

300/300 [==============================] - 1s 2ms/step


0.7166666674613953

In [40]:

prediction = model.predict_classes(test_images_arr)
pd.crosstab(y_test.iloc[:,1],prediction,
            rownames=['label'],colnames=['predict'])

predict,0,1,2,3
label,,,,
0,214,3,6,1
1,26,0,1,0
2,35,2,1,0
3,7,0,0,0
4,4,0,0,0


In [41]:
#### classification report

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix


names = ["stage 0", "stage 1", "stage 2", "stage 3", "stage 4"]
print(classification_report(y_test.iloc[:,1], prediction, target_names = names))
#print(confusion_matrix(y_test, prediction, labels=[0,1]))


              precision    recall  f1-score   support

     stage 0       0.75      0.96      0.84       224
     stage 1       0.00      0.00      0.00        27
     stage 2       0.12      0.03      0.04        38
     stage 3       0.00      0.00      0.00         7
     stage 4       0.00      0.00      0.00         4

    accuracy                           0.72       300
   macro avg       0.17      0.20      0.18       300
weighted avg       0.57      0.72      0.63       300



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
